# 6.5. Pomiar jakości klastrowania

W przypadku metod klastrowania, możemy rozpatrywać kwestię jakości na dwa sposoby. Jeśli znamy oczekiwany docelowy podział, to możemy sprawdzić czy wygenerowane klastry odpowiadają naszym oczekiwaniom. Jeśli jednak szukamy naturalnego podziału danych, ale nie zakładamy nic na temat podobieństwa poszczególnych obserwacji, to możemy rozpatrywać jakość w kategorii podobieństw wewnątrz klastrów.

## Jakość w przypadku nadzorowanym

Powróćmy do przypadku zbioru danych Titanic. Nauczmy jeszcze raz ten sam model co w poprzednim rozdziale i zbadajmy jaka jest jakość takiego rozwiązania.

In [1]:
import pandas as pd

In [2]:
reduced_titanic_df = pd \
    .read_parquet("../data/titanic-reduced.parquet")
reduced_titanic_df.sample(n=5)

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Master.,Dr.,Rev.,Names_count
758,1,3,20.0,0,0,7.2292,False,False,False,4
583,0,2,47.0,0,0,15.0000,False,False,False,4
265,1,3,25.0,1,0,7.7750,False,False,False,4
493,1,1,54.0,1,0,78.2667,False,False,False,4
56,0,3,28.5,0,0,7.2292,False,False,False,3


In [3]:
titanic_df = pd.read_parquet("../data/titanic-final.parquet")
titanic_df.sample(n=5)

,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Mr.,Miss.,Mrs.,Master.,Dr.,Rev.,Names_count,Sex_female
449,0,1,30.0,0,0,27.7500,True,False,False,False,False,False,4,0
589,0,3,47.0,0,0,7.2500,True,False,False,False,False,False,4,0
756,0,3,23.0,0,0,14.5000,True,False,False,False,False,False,3,0
744,0,1,19.0,1,0,53.1000,True,False,False,False,False,False,4,0
8,1,3,27.0,0,2,11.1333,False,False,True,False,False,False,7,1


### Ponowne nauczenie modelu

Stworzymy ten sam model co poprzednio, zanim jeszcze zaczniemy mierzyć jakość powstałych klastrów.

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [5]:
pipeline = Pipeline(steps=[
    ("scaling", StandardScaler()),
    ("clustering", KMeans(n_clusters=2, 
                          random_state=2019)),
])

In [6]:
import numpy as np

In [7]:
clusters = pipeline.fit_predict(
    reduced_titanic_df.drop(columns="Survived")
)

### Jednorodność (ang. *Homogeneity*)

Jednorodność jest miarą, która pozwala oszacować czy powstałe klastry są zgodne z oczekiwanym podziałem, tj. czy każdy klaster zawiera tylko elementy pojedynczej klasy. Miara ta jest wartością z przedziału $ [0, 1] $, gdzie $ 1 $ oznacza perfekcyjne dopasowanie.

In [8]:
from sklearn.metrics import homogeneity_score

In [9]:
homogeneity_score(reduced_titanic_df["Survived"], 
                  clusters)

0.0003201906697905245

### Kompletność (ang. *Completeness*)

Kompletność mierzy jakość klastrowania poprzez sprawdzenie czy wszystkie elementy należące do każdej klasy, należą równocześnie do tego samego klastra.

In [10]:
from sklearn.metrics import completeness_score

In [25]:
completeness_score(reduced_titanic_df["Survived"],
                   clusters)

0.0006315633833212408

### V-measure

Gdybyśmy wykorzystali średnią harmoniczną jednorodności oraz kompletności, to otrzymamy metrykę *V-measure*.

In [12]:
from sklearn.metrics import v_measure_score

In [13]:
v_measure_score(reduced_titanic_df["Survived"], clusters)

0.0004249431921190903

## Pomiar jakości, a uczenie nienadzorowane

Rozpatrując klastrowanie w uczeniu nadzorowanym, jesteśmy w stanie równie dobrze skorzystać z metryk charakterystycznych dla klasyfikacji, żeby zmierzyć skuteczość naszego rozwiązania. W sytuacji uczenia bez podanych etykiet, musimy do problemu podejść w inny sposób i oprzeć pomiar jakości na odległości poszczególnych obserwacji od siebie.

Metody te potrzebują do obliczeń zbioru danych, ale w takiej postaci, w jakiej jest on przekazany do metody klastrowania.

In [14]:
X = pipeline.named_steps["scaling"] \
    .transform(reduced_titanic_df.drop(columns="Survived"))

### Silhouette

Ta metoda pozwala na obliczenie tego, jak bardzo dany obiekt jest podobny do swojego klastra w porównaniu do innych klastrów. Metryka zwraca wartość w przedziale $ [-1, 1] $.

In [15]:
from sklearn.metrics import silhouette_score, \
    silhouette_samples

In [16]:
silhouette_samples(X, clusters)

array([ 0.54662522,  0.3688598 ,  0.60781224,  0.37453795,  0.61861495,
        0.60991318,  0.52206411,  0.29136251,  0.23347247,  0.47131705,
        0.31169601,  0.50434287,  0.59526378,  0.06301867,  0.50991968,
        0.53930959,  0.29831891,  0.60810446,  0.38133393,  0.59507   ,
        0.62746851,  0.61320217,  0.55207398,  0.54998116, -0.09523231,
        0.06318492,  0.61660068,  0.02848851,  0.60253186,  0.59916899,
        0.5610467 ,  0.35067477,  0.5832602 ,  0.51042495,  0.4730053 ,
        0.51398099,  0.57304306,  0.60032663,  0.38278148,  0.48852145,
        0.46226271,  0.31749495, -0.12856658,  0.58953041,  0.62188267,
        0.52586264,  0.60975156,  0.38996428,  0.48455288,  0.30665434,
        0.60026905,  0.42190782,  0.47116576,  0.42486704,  0.53697552,
        0.58396925,  0.61159004, -0.18095924,  0.33628239,  0.59507146,
        0.51365868,  0.48339678,  0.31818733,  0.49132998,  0.18056554,
        0.62482906,  0.5895913 ,  0.09212674,  0.43749707,  0.62

In [20]:
silhouette_score(X, clusters)

0.46973465065192194

Możemy też wykorzystać tę metrykę do tego, aby sprawdzić jak podobne byłyby do siebie wektory, jeśli odnaleźlibyśmy metodę, która przeprowadzi idealne klastrowanie pod kątem przetrwania katastrofy.

In [21]:
silhouette_score(X, reduced_titanic_df["Survived"])

0.07008673847575181

### Davies-Bouldin score

Metoda Davies-Bouldin score wylicza średnie podobieństwo pomiędzy danym klastrem, a najbardziej podobnym do niego innym klastrem. Podobieństwem, w tym wypadku, jest stosunek odległości wewnątrz klastra, do odległości pomiędzy tymi klastrami.

In [22]:
from sklearn.metrics import davies_bouldin_score

In [23]:
davies_bouldin_score(X, clusters)

1.4628528069110356

Zwracana wartość pochodzi z przedziału $ [0, \infty) $ i im mniejsza wartość, tym lepsze klastrowanie.